<a href="https://colab.research.google.com/github/ankitg-02/data_analysis/blob/main/Home_Office.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
home_office_sales=pd.read_excel("/content/Sample-sales-data-excel.xls")
count_home_office=0
for i in home_office_sales['Segment']:
  if i=='Home Office':
    count_home_office+=1
print("count of home offices=",count_home_office)
record_home_office=home_office_sales.groupby('Segment')
print(record_home_office.Category.get_group("Home Office").value_counts())

count of home offices= 1783
Office Supplies    1079
Furniture           362
Technology          342
Name: Category, dtype: int64
